# Evaluate and Optimize Abbreviation Extraction

In [3]:
import pandas as pd
import string

In [4]:
# Load Data
data = pd.read_csv('abbr_db.CSV', names=['abbr', 'long_forms'], sep=';', encoding='utf8')
abbr = list(data['abbr'].values)
expansions = list(data['long_forms'].values)

In [5]:
def upper_to_lower_ratio(w):
    upper_cases = ''.join([c for c in w if c.isupper()])
    return len(upper_cases)/len(w)

In [6]:
def extraction_rule(w, l_, ratio):
    w = w.replace("-", "")
    if len(w) <= l_ and upper_to_lower_ratio(w) >= ratio:
        return True
    else:
        return False

In [7]:
def determine_FN(l_, ratio):
    FN = 0
    for abb in abbr:
        if not extraction_rule(abb, l_, ratio):
            FN = FN + 1
    return FN

In [8]:
def determine_FP(l_, ratio):
    FP = 0
    for exp in expansions:
        for word in exp.split():
            if extraction_rule(word, l_, ratio):
                FP = FP + 1
    return FP

In [9]:
max_f1 = 0
memory = []
for l_ in range(1,21):
    for r in range(2, 98):
        r_ = r/100
        FP = determine_FP(l_, r_)
        FN = determine_FN(l_, r_)
        TP = len(abbr) - FN
        precision = TP/(TP + FP)
        recall = TP/(TP + FN)
        f1 = (2*recall*precision)/(recall + precision)
        if f1 > max_f1:
            max_f1 = f1
            memory = [l_, r, precision, recall, f1]
print(memory)

[13, 29, 0.92071468453378, 0.9232922732362822, 0.922001677383282]


In [81]:
FN = 0
for abb in abbr:
    if not extraction_rule(abb, 13, 0.29):
        FN = FN + 1
        print(abb)
print(str(FN))

AE lock
Alpha
A record
Auto
Auto
Auto
Basic
Bi-di
Bool
bps
Brouter
Cell phone
Cert
Cert
Cert
Char
chdir
Code
Codec
Codec
Coil
Core
Crypto
Ctrl
Curmbox
Double
Dpkg
Drop
Dword
E-911
E-book
E-learning
E-mail
Eval
Famicom
Feat
Fixed
Flash
Float
Fsck
Fstab
Fstype
Gbps
Gmail
Gravatar
Gzip
Hiew
Hi-res
Icacl
Inetd
Info
Inode
kb / kbit
Kbps
kilo
Laser
LED monitor
Li-ion
Li-polymer
Lisp
Lo-res / low-res
Luser
lvl
Lzone
m
Macro
Matlab
Matrix
mb
Mbps
Memo
Mflop
Mget
Mkdir
Moblog
Model no.
Modem
MO diskette
Mofo
Mput
Mutex
MX record
ne
Nimrod
nm
ns
Open
Parm
P-code
P-code
Permalink
Plus
P-machine
Printf
Privmsg
Proglog
Prolog
PrtScrn or Prt Scrn
Rec.
Regex
RF shielding
RLL encoding
RO terminal
Scanf
Serial no.
Servlet
Setuid
Sexp or sexpr
SHV connector
Slogin
Slotket
SM card
Source
Spec
Spit
splogs
Spool
Stdin
Subnet
sync
synth
sysgen
syslog
sysop
sysrq
Tech
Telco line
Texel
Transistor
Uarch
Uuencode
Veronica
Vhost
Web app
Winsock
Wpcomp
Wpcomp
137


In [2]:
for i in range(1,15):
    print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
